# Init

In [1]:
global_var = {

    # Connection
    'ip_address': '127.0.0.1',                  # Ip to connect
    'port_input': 8280,                         # Port where matlab send data
    'port_output': 8281,                        # Port where matlab receive data
    'port_f_input': 8282,                       # Same but for feedback branch
    'port_f_output': 8283,                      # Same but for feedback branch
    'buffer_size': 8,                           # Number of bytes to read 
    'connection_dim_input': 1,                  # Number of data to read -> used to unpack from byte, recevied data are byte
    'connection_dim_output': 3,                 # Number of data in output -> used to pack to byte and sedn to matlab
    'stop_flag': [-999, -999, -999, -999],      # DEPRECATED
    'use_pid': True,                            # DEPRECATED
        
    # Matlab
    'matlab_path': r'C:/Users/pc/Desktop/Artificial Intellingence & Robotics/1y-2s/Intelligent and hybrid control/IHC_attidute_control/control_schemas',
    'simulation_name': 'pid_implementation',
    'simulation_time': 10.0,


    # Network 
    'input_dim': 6,               # Dimension of input layer
    'output_dim': 3,              # Dimension of output layer
    'hidden_dim': 10,             # Dimension of hidden layer
    'bias': False,                
    'learning_rate': 0.001,       
    'model_name': 'model.pt',     # used to save weights

    #Train
    'batch_size': 32,
    'start_train_size': 1,
    'epochs': 10,               
    'epoch_size': 10,
    'train_frequency': 100
}

class Color:
    RED = '\033[91m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    BLUE = '\033[94m'
    MAGENTA = '\033[95m'
    CYAN = '\033[96m'
    WHITE = '\033[97m'
    RESET = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

# Imports

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import struct
import socket
import time
from simple_pid import PID

import matlab.engine
from multiprocessing import Process

# Architecture

In [3]:
class FeedForwardNet(nn.Module):

    def __init__(self, input_dim, output_dim, hidden_dim, bias ) -> None:
        super(FeedForwardNet, self).__init__()

        self.input_dim  = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim =  output_dim

        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()

        self.layer_1 = nn.Linear(
            in_features = self.input_dim,
            out_features = self.hidden_dim,
            bias = bias
        )

        self.layer_2 = nn.Linear(
            in_features = self.hidden_dim,
            out_features = self.hidden_dim,
            bias = bias
        )

        self.layer_3 = nn.Linear(
            in_features = self.hidden_dim,
            out_features = self.hidden_dim,
            bias = bias
        )

        self.layer_4 = nn.Linear(
            in_features = self.hidden_dim,
            out_features = self.output_dim,
            bias = bias
        )
    
    def forward(self, x):
        
        x = self.layer_1(x)
        
        x = self.layer_2(x) 
        x = self.tanh(x)
        x = self.layer_3(x) 
        x = self.tanh(x)
        x = self.layer_4(x)
        
        x = 0.5*(1+self.tanh(x))

        return x

In [4]:
class ControllerNetwork(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim, lr, bias, model_name ) -> None:
        super(ControllerNetwork, self).__init__()

        self.network = FeedForwardNet(
            input_dim = input_dim,
            output_dim = output_dim,
            hidden_dim = hidden_dim,
            bias = bias
        )

        self.model_name = model_name
        self.loss_fnc = nn.MSELoss()
        self.optimizer = optim.Adam(self.parameters(), lr=lr)

    def forward(self, x):
        network_output = self.network(x)
        return network_output
    
    def save(self):
        name = self.model_name
        torch.save(self.state_dict(), name )
        #print(f"{Color.MAGENTA}Saved: {name}{Color.RESET}")

    def load(self):
        name = self.model_name
        try:
            self.load_state_dict(torch.load(name) )
            print(f"{Color.MAGENTA}loaded: {name}{Color.RESET}")
        except Exception as e:
            print(f"{Color.RED}Model not loaded{Color.RESET}")
            print(e)

# Utils


In [5]:
def setup_socket(ip_address, port_input):

    print(f"{Color.YELLOW}ip: {ip_address}, port_input: {port_input}{Color.RESET}")

    try:
        socket_nn_input = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        socket_nn_input.bind( (ip_address, port_input) )
        socket_nn_input.listen(1)

        print(f"{Color.BOLD}{Color.GREEN}Sockets listener created{Color.RESET}")

        return socket_nn_input
    except Exception as e:
        print(f"{Color.RED}An error occurred: {e}{Color.RESET}")

        if 'socket_nn_input' in locals():
            socket_nn_input.close()
            
        return None, None
    
def accept_connection(socket_server):
    connection, address = socket_server.accept()
    print(f"{Color.GREEN}Connection accepted!{Color.RESET}")
    return connection, address

def receive_data(connection, buffer_size, dim_input):
    
    expected_bytes = buffer_size  # Size for one double
    no_data_flag = False
    data = b''
    while len(data) < expected_bytes:
        more_data = connection.recv(expected_bytes - len(data))

        if not more_data:
            print("\nNo data received. Ending connection.")
            no_data_flag = True
            break

        data += more_data
    data = list(struct.unpack(f'!{str(dim_input)}d', data))  # Unpack one double
    
    return data, no_data_flag

def receive_data_excpt(connection, buffer_size, dim_input, stop_flag):
    
    expected_bytes = 8# buffer_size  # Size for one double
    data = b''
    #connection.settimeout(5.0)  # Set timeout to 5 seconds

    try:
        while len(data) < expected_bytes:
            #more_data = connection.recv(expected_bytes - len(data))
            more_data = connection.recv(expected_bytes)
            if not more_data:
                # No more data is available, break the loop
                break
            data += more_data
        try:
            #data = list(struct.unpack(f'!{str(dim_input)}d', data)) 
            data = list(struct.unpack(f'!d', data)) 
        except Exception as e:
            print(f"\n{e}")
            print(f"{Color.RED}\nProblem with unpacking, error: {e}{Color.RESET}")
            print(f"{Color.RED}May be due because return empty string when nothing is receive for a certain time{Color.RESET}")
            return stop_flag
        
    except Exception as e:

        print(f"\n{e}")
        if isinstance(e, socket.timeout):
            print(f"{Color.RED}\nTimeout error: No data received within the timeout period{Color.RESET}")
        else:
            print(f"{Color.RED}\nOther exception occurred: {e}{Color.RESET}")
            print(f"{Color.RED}Maybe due to some other error in the code{Color.RESET}")
            
        print(f"{Color.RED}May be due because return empty string when nothing is receive for a certain time{Color.RESET}")
        print(f"{Color.BLUE}\nNo data received within the timeout period, maybe some error of code{Color.RESET}")
       
        return stop_flag
    
    connection.settimeout(None)
    return data

def send_data(connection, message, dim_output):
    try:
        #message_to_send = struct.pack(f'!{str(dim_output)}d', *message)  
        message_to_send = struct.pack(f'!{str(dim_output)}d', *message)
        connection.sendall(message_to_send)

    except Exception as e:
        print(f"\rError sending float: {e}", end='')

def close_connections(*to_close):
    try:
        for c in to_close:
            c.close()
    except Exception as e:
        print(f"{Color.RED}\nFailed to close connection: verified the follwing error:\n{e}{Color.RESET}")
        return

    print(f"{Color.GREEN}\nSockets closed!{Color.RESET}")



In [6]:
def run_simulink(model_name, matlab_path, simulation_time):
    print(f"Starting Matlab engine ... ", end='')
    future = matlab.engine.start_matlab(background=True)
    eng = future.result()  
    print(f"Started!")

    eng.cd(matlab_path)
    print("MATLAB working directory:", eng.pwd())

    eng.load_system(model_name)
    loaded_model = eng.bdroot()
    print(f"The currently loaded model is: {loaded_model}")

    eng.set_param(model_name, 'StartTime', '0', nargout=0)
    eng.set_param(model_name, 'StopTime', str(simulation_time), nargout=0)
    eng.set_param(model_name, 'SimulationCommand', 'start', nargout=0)

    return eng

def run_simulink_with_gui(model_name, matlab_path, simulation_time):
    print(f"Starting Matlab engine ... ", end='')
    future = matlab.engine.start_matlab(background=True)
    eng = future.result()  
    print(f"Started!")

    eng.cd(matlab_path)
    print("MATLAB working directory:", eng.pwd())

    eng.open_system(model_name, nargout=0)
    loaded_model = eng.bdroot()

    print(f"The currently loaded model is: {loaded_model}")

    eng.set_param(model_name, 'StartTime', '0', nargout=0)
    eng.set_param(model_name, 'StopTime', str(simulation_time), nargout=0)
    eng.set_param(model_name, 'SimulationCommand', 'start', nargout=0)

    return eng

# Dataset

In [7]:
class ControllDataset(Dataset):
    def __init__(self):
        self.data = []

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        state, label = item[:-1], item[-1]
        return state, label

    def add_data(self, new_data):
        self.data.append(new_data)

# Train

In [8]:
def train(model, dataset, n_epochs, epoch_size, batch_size, shuffle):

    model.train()
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    
    iteration_epoch = 0
    for e in range(n_epochs):
        #print(f"\r it: {it}")
        loss_value = 0
        iteration_epoch = 0

        for states, targets in dataloader:

            iteration_epoch += 1
            if iteration_epoch >= epoch_size:
                break

            model.optimizer.zero_grad()
            
            targets = targets.unsqueeze(1)
            
            _ = model(states)

            zeros_tensor = torch.zeros(targets.shape)
            
            l2_lambda = 0.001  # Example value for L2 regularization factor
            l2_norm = sum(p.pow(2.0).sum() for p in model.parameters())
            l2_reg = l2_lambda*l2_norm

            loss = model.loss_fnc( zeros_tensor, targets ) #+ l2_reg
            loss.backward()
            

            model.optimizer.step()

            loss_value += loss.item()

            break

        model.save()
        
    model.eval()
    return loss.mean()

# Main

In [9]:
network = ControllerNetwork(  
                            input_dim=global_var['input_dim'],
                            output_dim=global_var['output_dim'],
                            hidden_dim=global_var['hidden_dim'],
                            bias=global_var['bias'],
                            lr=global_var['learning_rate'],
                            model_name=global_var['model_name']
                            )

pid = PID(2.0, 2.0, 2.0, setpoint=1) # random initialization

In [10]:
receiver_socket = setup_socket(
    global_var['ip_address'], 
    global_var['port_input']
    )

send_socket = setup_socket(
    global_var['ip_address'], 
    global_var['port_output']
    )


eng_path = r'C:/Users/pc/Desktop/Artificial Intellingence & Robotics/1y-2s/Intelligent and hybrid control/IHC_attidute_control/control_schemas'
model_name = 'pid_implementation'
simulation_time = 10.0  # Define your simulation time
    

#eng = run_simulink_with_gui(model_name, eng_path, simulation_time)
eng = run_simulink(
    model_name=global_var['simulation_name'], 
    matlab_path=global_var['matlab_path'], 
    simulation_time=global_var['simulation_time']
    )


print(f"{Color.CYAN}\nWaiting someone to connect ...{Color.RESET}")
connection_receiver, addr = accept_connection(receiver_socket)
connection_sender, addr = accept_connection(send_socket)




ip: 127.0.0.1, port_input: 8280
Sockets listener created
ip: 127.0.0.1, port_input: 8281
Sockets listener created
Starting Matlab engine ... Started!
C:/Users/pc/Desktop/Artificial Intellingence & Robotics/1y-2s/Intelligent and hybrid control/IHC_attidute_control/control_schemas
MATLAB working directory: C:\Users\pc\Desktop\Artificial Intellingence & Robotics\1y-2s\Intelligent and hybrid control\IHC_attidute_control\control_schemas
The currently loaded model is: pid_implementation

Waiting someone to connect ...
Connection accepted!
Connection accepted!


In [11]:
# https://simple-pid.readthedocs.io/en/latest/reference.html

dataset = ControllDataset()
n_received = 0
n_sent = 0
previous_error = 0

# need to send one data in order to initialize 
# block which wait data from python 

send_data(
        connection=connection_sender, 
        message=[0.0, 0.0, 0.0], 
        dim_output=global_var['connection_dim_output']
        ) 

first_sample = True
previous_sample = None

pid.tunings = (5.0, 2.0, 1.0)

In [12]:
#network.load()

print(f"\rreceived: {n_received}, sent; {n_sent}", end="")
pid_parameters = [pid.tunings[0], pid.tunings[1], pid.tunings[2]]  # dummy
mean_loss = 0

try:
    while True:

        raw_data, no_data_flag = receive_data(
            connection=connection_receiver, 
            buffer_size=global_var['buffer_size'],
            dim_input=global_var['connection_dim_input'],
            #stop_flag=global_var['stop_flag']
            )
        
        n_received += 1
        
        '''
        if no_data_flag == True:
            print("\nNo data received. Ending connection.")
            break
        

        if raw_data[0] == global_var['stop_flag'][0]:
            break

        
        raw_data = [raw_data[0], raw_data[1], raw_data[2], pid.tunings[0], pid.tunings[1], pid.tunings[2], raw_data[3]]
        #print(raw_data)
        raw_data = torch.tensor(raw_data)
        mean = raw_data.mean(dim=0)
        std = raw_data.std(dim=0)

        # Normalize the data
        normalized_data_tensor = (raw_data - mean) / std

        
        if first_sample == False:
            #previous_sample[-1] = raw_data[-1]
            #previous_sample[-1] = normalized_data_tensor[-1]
            data = torch.tensor(previous_sample, requires_grad=True)
            dataset.add_data(data)

        previous_sample = raw_data
        first_sample = False 
        
        #data = torch.tensor(raw_data, requires_grad=True)
        #dataset.add_data(data)
        
        if ( len(dataset) % 32 ) == 0 and len(dataset) > global_var['start_train_size']:
            
            current_loss = train(
                model=network,
                dataset=dataset,
                n_epochs=global_var['epochs'],
                epoch_size=global_var['epoch_size'],
                batch_size=global_var['batch_size'],
                shuffle=True
            ) 

            mean_loss = (current_loss + mean_loss).mean()
        
        
        network_input = raw_data[:-1]
        error = raw_data[-1]

        net_output = network(network_input)
        net_output = net_output.detach()

        pid.setpoints = raw_data[1]

        if len(dataset) > global_var['start_train_size']:
            pid_parameters = net_output.numpy().tolist()
            pid.tunings = (pid_parameters[0], pid_parameters[1], pid_parameters[2])
        '''
        
        #output = [pid(error)] 
        output = [5.0, 2.0, 2.0]

        send_data(
            connection=connection_sender, 
            message=output, 
            dim_output=global_var['connection_dim_output']
            )
        n_sent += 1
        
        print(f"\rreceived: {n_received}, sent: {n_sent}, n_data: {len(dataset)}, pid_parameters: ({pid_parameters[0]:.4f}, {pid_parameters[1]:.4f}, {pid_parameters[2]:.4f}),  error: {raw_data[-1]:.2f}, mean loss: {mean_loss:.2f}", end="")

finally:

    close_connections(
        connection_receiver,
        connection_sender,
        send_socket,
        receiver_socket,
    )

    eng.quit()


received: 14028, sent: 14028, n_data: 0, pid_parameters: (5.0000, 2.0000, 1.0000),  error: 0.00, mean loss: 0.000
No data received. Ending connection.

Sockets closed!


error: unpack requires a buffer of 8 bytes